# Dataset Processor Notebook

This notebook processes a dataset by comparing clothing descriptors.

In [1]:
!which python

/apps/jupyter/6.5.4/bin/python


In [2]:
# ...existing imports...
import os
import shutil
import cv2
import numpy as np

In [8]:
# Threshold for descriptor similarity (tune as needed)
DESCRIPTOR_DISTANCE_THRESHOLD = 0.3

def detect_person(image):
    """
    Use the HOG+SVM detector to find and return the bounding box of the largest detected person.
    Returns the bounding box [x, y, w, h] or None if no person is detected.
    """
    hog = cv2.HOGDescriptor()
    hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
    (rects, _) = hog.detectMultiScale(image, winStride=(4, 4), padding=(8, 8), scale=1.05)
    if len(rects) == 0:
        return None

    # Choose the largest detection (by area)
    rect = max(rects, key=lambda r: r[2] * r[3])
    return rect


In [17]:
def compute_descriptor(image):
    """
    Compute a descriptor for the entire image by combining:
      - HOG features computed on a resized version of the image.
      - A normalized HSV color histogram computed on the full image.
      
    This descriptor is used to compare if two frames have a similar overall appearance.
    """
    # Resize image to a fixed size for the HOG feature extraction.
    resized_image = cv2.resize(image, (128, 256))
    
    # Compute HOG features.
    hog = cv2.HOGDescriptor()
    hog_features = hog.compute(resized_image)
    hog_features = hog_features.flatten()

    # Compute a normalized color histogram in HSV space from the full image.
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, [8, 8, 8], [0, 180, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    color_hist = hist.flatten()

    # Concatenate the HOG features and color histogram
    descriptor = np.concatenate([hog_features, color_hist])
    return descriptor

In [18]:
def get_frame_descriptor(frame_path):
    """
    Reads an image from the given frame path and returns its descriptor.
    """
    image = cv2.imread(frame_path)
    if image is None:
        print(f"Error reading {frame_path}")
        return None
    descriptor = compute_descriptor(image)
    if descriptor is None:
        print(f"Could not compute descriptor for {frame_path}.")
    return descriptor

In [19]:
def group_frames(frame_paths):
    """
    Compares frame descriptors and groups frames that appear to be similar.
    Returns a dictionary:
       key: group id, 
       value: list of tuples (frame_path, descriptor)
    """
    groups = {}  # group id -> list of (frame_path, descriptor)
    next_group = 1

    for path in frame_paths:
        descriptor = get_frame_descriptor(path)
        if descriptor is None:
            continue

        assigned = False
        for group_id, members in groups.items():
            rep_descriptor = members[0][1]
            distance = np.linalg.norm(descriptor - rep_descriptor)
            if distance < DESCRIPTOR_DISTANCE_THRESHOLD:
                groups[group_id].append((path, descriptor))
                assigned = True
                break

        if not assigned:
            groups[next_group] = [(path, descriptor)]
            next_group += 1

    return groups

In [20]:
def show_comparison(img_path1, img_path2):
    """
    Displays two images side by side for visual comparison.
    """
    img1 = cv2.imread(img_path1)
    img2 = cv2.imread(img_path2)
    if img1 is None or img2 is None:
        print("Error reading images for comparison.")
        return
    h = 400
    img1 = cv2.resize(img1, (int(img1.shape[1] * h / img1.shape[0]), h))
    img2 = cv2.resize(img2, (int(img2.shape[1] * h / img2.shape[0]), h))
    combined = np.hstack((img1, img2))
    cv2.imshow("Comparison (Left: Group 1, Right: Group 2)", combined)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [21]:
def process_camera(camera_path):
    """
    Processes a camera folder by:
      - Checking for existence of a 'frames' subdirectory.
      - Grouping frames based on appearance descriptors computed on the whole image.
      - If multiple groups are detected, letting the user choose which group to keep.
      - Deleting frames (or the folder if no PNG files exist) accordingly.
    """
    frames_dir = os.path.join(camera_path, "frames")
    if not os.path.exists(frames_dir):
        print(f"No frames directory in {camera_path}. Deleting the camera folder.")
        # shutil.rmtree(camera_path)
        return

    frame_files = [os.path.join(frames_dir, f) for f in os.listdir(frames_dir) if f.endswith(".png")]
    if not frame_files:
        print(f"No PNG frames in {frames_dir}. Deleting the camera folder {camera_path}.")
        # shutil.rmtree(camera_path)
        return

    groups = group_frames(frame_files)

    if len(groups) <= 1:
        print(f"All frames in {frames_dir} appear to have a similar overall appearance.")
        return

    print(f"Detected {len(groups)} groups in {frames_dir}.")

    # If there are exactly two groups, display them side by side.
    if len(groups) == 2:
        group_ids = sorted(groups.keys())
        rep1 = groups[group_ids[0]][0][0]
        rep2 = groups[group_ids[1]][0][0]
        show_comparison(rep1, rep2)
        choice = input("Which group should be kept? Type 1 for left image or 2 for right image: ").strip()
        try:
            keep_group = group_ids[int(choice) - 1]
        except (ValueError, IndexError):
            print("Invalid choice. Skipping deletion for this folder.")
            return

        for gid, items in groups.items():
            if gid != keep_group:
                for (fp, _) in items:
                    print(f"Deleting {fp}")
                    # os.remove(fp)
    else:
        # For more than 2 groups, list the representative images and ask the user which group to keep.
        print("More than two groups detected. Please choose which group to keep:")
        rep_images = {}
        for gid, items in groups.items():
            rep_images[gid] = items[0][0]
            print(f" Group {gid}: {rep_images[gid]}")
        choice = input("Enter the group number you want to keep: ").strip()
        try:
            keep_group = int(choice)
        except ValueError:
            print("Invalid choice. Skipping deletion.")
            return

        if keep_group not in groups:
            print("Chosen group does not exist. Skipping deletion.")
            return

        for gid, items in groups.items():
            if gid != keep_group:
                for (fp, _) in items:
                    print(f"Deleting {fp}")
                    # os.remove(fp)

In [22]:
def main(root_dir):
    """
    Main function to iterate over subject subdirectories and process each camera folder.
    """
    if not os.path.isdir(root_dir):
        print(f"{root_dir} is not a directory")
        return

    # Iterate over subject subdirectories
    for subject in os.listdir(root_dir):
        subject_path = os.path.join(root_dir, subject)
        if not os.path.isdir(subject_path):
            continue

        # Iterate over camera directories in each subject
        for camera in os.listdir(subject_path):
            camera_path = os.path.join(subject_path, camera)
            if not os.path.isdir(camera_path):
                continue

            # For example, if you want to remove folders named "flir".
            if camera.lower() == "flir":
                print(f"Removing camera folder {camera_path}")
                # shutil.rmtree(camera_path)
                continue

            process_camera(camera_path)

In [23]:
# Execute the main function
root_dir = input('Enter the root dataset directory: ').strip()
main(root_dir)

Enter the root dataset directory:  /home/caio.dasilva/Vibe3D/kitwarebrc/output


KeyboardInterrupt: 